In [2]:
from gensim.models import KeyedVectors

In [3]:
# file_path = 'nkjp+wiki-forms-all-100-skipg-hs.txt.gz'

# model = KeyedVectors.load_word2vec_format(file_path, binary=False)

In [4]:
file_path = 'nkjp+wiki-forms-all-100-skipg-ns.txt.gz'

model = KeyedVectors.load_word2vec_format(file_path, binary=False)

In [5]:
import json
from pathlib import Path

In [6]:
json_files = list(Path("./data/ground_truth").rglob("*.jsonl"))

posts = []

for file in json_files:
    with open(file, 'r') as f:
        data = [json.loads(line) for line in f]
        posts.extend(data)

In [7]:
sentence_labels = ['Mowa nienawiści', 'Neutralny', 'Hate', 'Neutral']

In [8]:
def get_labels(text, labels):
    labeled_texts = []
    word_labels = [label for label in labels if label[2] not in sentence_labels]
    for label in word_labels:
        labeled_text = text[label[0]: label[1]]
        labeled_texts.append((labeled_text, label[2]))
    return labeled_texts

In [9]:
all_words = []

for post in posts:
    text = post["text"]
    labels = get_labels(post["text"], post["label"])
    words = text.split(" ")

    for word in words:
        new_word = {"name": word, "label": None, "embedding": None}
        if word in model:
            new_word["embedding"] = model[word]

        for label in labels:
            if word in label[0]:
                new_word["label"] = label[1]
        all_words.append(new_word)

In [10]:
import numpy as np

In [11]:
emb_tensor = [word["embedding"] for word in all_words if word["embedding"] is not None]
emb_tensor = np.stack(emb_tensor)

In [12]:
from sklearn.manifold import TSNE

In [13]:
tsne = TSNE()

In [14]:
clustered = tsne.fit_transform(emb_tensor)

In [15]:
import pandas as pd

In [16]:
df = pd.DataFrame(clustered)
df["label"] = [word["name"] for word in all_words if word["embedding"] is not None]
df["sentiment"] = [word["label"] for word in all_words if word["embedding"] is not None]
df

,0,1,label,sentiment
0,51.256546,-19.382858,gdyby,None
1,30.061703,33.152561,to,None
2,87.582817,-1.168580,była,None
3,29.352610,35.406845,to,None
4,56.823418,-20.397739,by,None
...,...,...,...,...
3262,59.106426,-26.319626,chciał,None
3263,26.543854,-29.754601,pałować,None
3264,-22.962851,-103.143585,nie,None
3265,30.558128,-57.293434,bądźmy,None


In [17]:
import plotly.express as px

In [21]:
df['sentiment'] = df['sentiment'].fillna('Brak')

In [23]:
fig = px.scatter(df, x=0, y=1, hover_data=df.columns, color="sentiment")
fig.show()